In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns                                                                                                                                                                                                                          
import pandas as pd
import numpy as np

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pickle
import yfinance as yahooFinance
from sklearn import tree
import os
import plotly
import plotly.graph_objs as go
import ffn
from jinja2 import Environment, FileSystemLoader

from sklearn.tree import _tree

import graphviz

In [2]:
def get_weekly(instrument):
    # SPY = yahooFinance.Ticker("SPY")
    SYM = yahooFinance.Ticker(instrument)
    sym = SYM.history(period="max")

    logic = {'Open'  : 'first',
             'High'  : 'max',
             'Low'   : 'min',
             'Close' : 'last',
             'Volume': 'sum'}
    

    
    weekly  = sym.resample('W-Fri',).apply(logic).reset_index()
    
    sym = sym.reset_index()[['Date','Open','High','Low','Close','Volume']]
    
#     weekly_with_indicators = dataset_with_indicators(weekly)
#     weekly_with_indicators['%change']=weekly_with_indicators['Close'].pct_change()    
    return weekly

In [328]:
def generate_report():
    class PerformanceReport:
        """ Report with performance stats for given strategy returns.
        """

        def __init__(self,infilename):
            self.infilename = infilename
            self.get_data()

        def get_data(self):
            basedir = os.path.abspath(os.path.dirname('__file__'))
            data_folder = os.path.join(basedir, 'data')
            data = pd.read_csv('res1.csv', index_col = 'date',parse_dates = True,dayfirst = True) 
            #os.path.join(data_folder, self.infilename),index_col='date',

            #parse_dates=True,dayfirst=True)
            self.equity_curve = data['equity_curve']

            if len(data.columns) > 1:
                self.benchmark_curve = data['benchmark']

        def generate_html(self):
            env = Environment(loader=FileSystemLoader('.'))
            template = env.get_template("template (1).html")
            perf_chart = self.plot_performance_chart()
            drawdown_chart = self.plot_drawdown_chart()
            monthly_table = self.create_monthly_table(self.equity_curve.pct_change().dropna(),1)
    #         print(self.equity_curve)
            equity_curve_ffn_stats = self.get_ffn_stats(self.equity_curve)
            benchmark_curve_ffn_stats = self.get_ffn_stats(self.benchmark_curve)
            kpi_table = self.create_kpi_table(equity_curve_ffn_stats)
            kpi_table_full =  self.create_kpi_table_full([equity_curve_ffn_stats,benchmark_curve_ffn_stats])
            print(kpi_table_full)
            kpi_table_full = kpi_table_full.rename(index={'yearly_vol': "yearly volatility", "avg_drawdown_days": "avg drawdown duration in days", "mtd": "month to date","monthly_vol":"monthly volatility","incep":"inception"})
            print(kpi_table_full)

            kpi_table_full.iloc[3:6] = ((kpi_table_full.iloc[3:6].astype(float)*100).round(1)).astype('str')+'%'
            kpi_table_full.iloc[7:16] = ((kpi_table_full.iloc[7:16].astype(float)*100).round(1)).astype('str')+'%'
            kpi_table_full.iloc[26:28] = ((kpi_table_full.iloc[26:28].astype(float)*100).round(2)).astype('str')+'%'
            kpi_table_full.iloc[30:32] = ((kpi_table_full.iloc[30:32].astype(float)*100).round(2)).astype('str')+'%'
    #         kpi_table_full.iloc[36] = ((kpi_table_full.iloc[36:37].astype(float)*100).round(2)).astype('str')+'%'
    #         kpi_table_full.iloc[40] = ((kpi_table_full.iloc[40].astype(float)).round(2)*100).astype('str')+'%'
            kpi_table_full.iloc[38:41] = ((kpi_table_full.iloc[38:41].astype(float)*100).round(2)).astype('str')+'%'
            kpi_table_full.iloc[42:48] = ((kpi_table_full.iloc[42:48].astype(float)*100).round(2)).astype('str')+'%'
            kpi_table_full.iloc[34:36] = ((kpi_table_full.iloc[34:36].astype(float)*100).round(2)).astype('str')+'%'


            kpi_table_1,kpi_table_2,kpi_table_3,kpi_table_4,kpi_table_5 = self.split_kpi_table(kpi_table_full)
            daily_ret_hist = self.plot_daily_histogram()
            daily_ret_box = self.plot_daily_box()
            html_out = template.render(perf_chart=perf_chart,drawdown_chart=drawdown_chart,monthly_table=monthly_table,
                                        kpi_table=kpi_table,kpi_table_1=kpi_table_1,kpi_table_2=kpi_table_2,
                                        kpi_table_3=kpi_table_3,kpi_table_4=kpi_table_4,kpi_table_5=kpi_table_5,
                                        daily_ret_hist=daily_ret_hist,daily_ret_box=daily_ret_box)
            return html_out

        def generate_html_report(self):
            """ Returns HTML report with analysis
            """
            html = self.generate_html()
            outputdir="output"
            outfile = os.path.join(outputdir, 'report2.html')        
            file = open(outfile,"w")  
            file.write(html)
            file.close()      

        def rebase_series(self,series):
            return (series/series.iloc[0]) * 100   

        def plot_performance_chart(self):

            #plotly combined equity chart
            trace_equity = go.Scatter(
                                x=self.equity_curve.index.tolist(),
                                y=self.rebase_series(self.equity_curve).values.tolist(),
                                name='Strategy',
                                yaxis='y2',
                                line = dict(color = ('rgb(22, 96, 167)')))

            trace_benchmark = go.Scatter(
                                x=self.benchmark_curve.index.tolist(),
                                y=self.rebase_series(self.benchmark_curve).values.tolist(),
                                name='Benchmark',
                                yaxis='y2',
                                line = dict(color = ('rgb(22, 96, 0)')))

            layout = go.Layout(
                             autosize=True,
                             height = 450,
                             legend=dict(orientation="h"),
                             title='Performance Chart',
                             yaxis=dict(
                                 title='Performance'))

            perf_chart = plotly.offline.plot({"data": [trace_equity,trace_benchmark],
                                         "layout": layout}, include_plotlyjs=False,
                                             output_type='div')

            return perf_chart

        def plot_drawdown_chart(self):
            dr = pd.DataFrame()
            dr['date'] = self.equity_curve.to_drawdown_series().index.tolist()
            dr['equity_curve'] = self.equity_curve.to_drawdown_series().values
            dr['benchmark'] = self.benchmark_curve.to_drawdown_series().values
            dr.to_csv('drawdowns.csv')
            #plotly combined drawdown chart
            trace_equity_drawdown = go.Scatter(
                                x=self.equity_curve.to_drawdown_series().index.tolist(),
                                y=self.equity_curve.to_drawdown_series().values.tolist(),
                                name='Strategy',
                                yaxis='y2',
                                line = dict(color = ('rgb(22, 96, 167)')))

            trace_benchmark_drawdown = go.Scatter(
                                x=self.benchmark_curve.to_drawdown_series().index.tolist(),
                                y=self.benchmark_curve.to_drawdown_series().values.tolist(),
                                name='Benchmark',
                                yaxis='y2',
                                line = dict(color = ('rgb(22, 96, 0)')))

            layout = go.Layout(

                             autosize=True,
                             height = 600,
                             legend=dict(orientation="h"),
                             title='Drawdown Chart',
                             yaxis=dict(
                                 title='Drawdown'))

            drawdown_chart = plotly.offline.plot({"data": [trace_equity_drawdown,trace_benchmark_drawdown],
                                         "layout": layout}, include_plotlyjs=False,
                                             output_type='div')

            return drawdown_chart

        def create_monthly_table(self,return_series,num_of_compenents):
            return_series.rename('weighted rets',inplace=True)
            return_series = (return_series/ float(num_of_compenents))
            returns_df_m = pd.DataFrame((return_series + 1).resample('M').prod() - 1)
            returns_df_m['Month'] = returns_df_m.index.month
            monthly_table = returns_df_m[['weighted rets','Month']].pivot_table(returns_df_m[['weighted rets','Month']], index=returns_df_m.index, columns='Month', aggfunc=np.sum).resample('A')
            monthly_table = monthly_table.aggregate('sum')
            monthly_table.columns = monthly_table.columns.droplevel()
            #replace full date in index column with just the correspnding year
            monthly_table.index = monthly_table.index.year
            monthly_table['YTD%'] = ((monthly_table + 1).prod(axis=1) - 1)

            monthly_table = monthly_table * 100

#             monthly_table['SPY%'] = [1.15,37.22,22.74,33.06,28.35,20.86,-9.15,-11.86,-22.12,28.39,10.75,4.79,15.69,5.39,-36.97,26.42,14.93,1.68,14.21,31.44,11.39,-0.73,11.80,19.42,-4.45,31.29,18.25,15.25]
    #         monthly_table['%Days leveraged'] = [6,24.0, 24.0, 18.0, 2.0, 20.0, 6.0,0,0, 32.0, 12.0, 12.0, 18.0, 12.0, 0,20.0,
    #  14.0,
    #  14.0,
    #  22.0,
    #  30.0,
    #  16.0,
    #  10.0,
    #  26.0,
    #  32.0,
    #  30.0,
    #  14.0,
    #  28.0,
    #  18.0]
    #         monthly_table['%Days in cash'] = c

            #Replace integer column headings with MMM format
            monthly_table.columns = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Strategy YTD%'] #,'Benchmark SPY%'] #,'%Days in Cash']
            monthly_table.round(2).fillna("").to_csv('monthly_table.csv')
            return monthly_table.round(2).fillna("").to_html(classes="table table-hover table-bordered table-striped")

        def get_ffn_stats(self,equity_series):
            equity_stats = equity_series.calc_stats()
            d = dict(equity_stats.stats)
            return d

        def create_kpi_table(self,ffn_dict):
            kpi_table = pd.DataFrame.from_dict(ffn_dict,orient='index')
            kpi_table.index.name = 'KPI'
            kpi_table.columns = ['Value']
            kpi_table2 = kpi_table.loc[['total_return','cagr',
                                       'yearly_vol','max_drawdown','avg_drawdown']]#.astype(float)
            kpi_table2 = kpi_table2.rename(index={'yearly_vol': "yearly volatility", "avg_drawdown_days": "avg drawdown duration in days"})


            kpi_table2['Value'] = pd.Series(["{0:.2f}%".format(val * 100) for val in kpi_table2['Value']], index = kpi_table2.index)

            kpi_table2.loc['avg_drawdown_days'] = kpi_table.loc['avg_drawdown_days']
    #         kpi_table2.loc['daily_sharpe'] = np.round(kpi_table.loc['daily_sharpe'].values[0],2)
            return kpi_table2.to_html(classes="table table-hover table-bordered table-striped",header=False)

        def create_kpi_table_full(self,ffn_dict_list):
            df_list =[pd.DataFrame.from_dict(x,orient='index') for x in ffn_dict_list]
            kpi_table_full = pd.concat(df_list,axis=1)

            kpi_table_full.columns = ['Strategy','Benchmark SPY']

            return kpi_table_full

        def split_kpi_table(self,kpi_table_full):
            kpi_table_1 = kpi_table_full.iloc[3:16].to_html(classes="table table-hover table-bordered table-striped",header=True)
            kpi_table_2 = kpi_table_full.iloc[16:24].to_html(classes="table table-hover table-bordered table-striped",header=True)
            kpi_table_3 = kpi_table_full.iloc[24:32].to_html(classes="table table-hover table-bordered table-striped",header=True)
            kpi_table_4 = kpi_table_full.iloc[32:40].to_html(classes="table table-hover table-bordered table-striped",header=True)
            kpi_table_5 = kpi_table_full.iloc[40:].to_html(classes="table table-hover table-bordered table-striped",header=True)
            return kpi_table_1,kpi_table_2,kpi_table_3,kpi_table_4,kpi_table_5

        def plot_daily_histogram(self):

            trace0 = go.Histogram(x=self.equity_curve.pct_change().values,
                                  name="Strategy",
                                  opacity=0.75,
                                  marker=dict(
                                  color=('rgb(22, 96, 167)')),
                                  xbins=dict(
                                  size=0.0025
                                          ))

            trace1 = go.Histogram(x=self.benchmark_curve.pct_change().values,
                                  name="Benchmark SPY",
                                  opacity=0.75,
                                  marker=dict(
                                  color=('rgb(22, 96, 0)')),
                                  xbins=dict(
                                  size=0.0025
                                          ))

            data = [trace0,trace1]

            layout = go.Layout(
                                title='Histogram of Strategy and Benchmark Daily Returns',
                                autosize=True,
                                height=600,
                                hovermode = 'closest',
                                barmode='overlay'
                                )

            daily_ret_hist = plotly.offline.plot({"data": data,"layout": layout}, 
                                                  include_plotlyjs=False,
                                                  output_type='div')

            return daily_ret_hist


        def plot_daily_box(self):
            trace0 = go.Box(y=self.equity_curve.pct_change().values,
                            name="Strategy",
                            marker=dict(
                                         color=('rgb(22, 96, 167)')))

            trace1 = go.Box(y=self.benchmark_curve.pct_change().values,
                            name="Benchmark",
                            marker=dict(
                                  color=('rgb(22, 96, 0)')))

            data = [trace0,trace1]

            layout = go.Layout(
                             title='Boxplot of Strategy and Benchmark Daily Returns',
                             autosize=True,
                             height=600,
                             yaxis=dict(
                                    zeroline=False
                                )
                             )

            box_plot = plotly.offline.plot({"data": data,"layout": layout}, include_plotlyjs=False,
                                             output_type='div')

            return box_plot


    report = PerformanceReport('res1.csv')
    report.generate_html_report()

In [329]:
generate_report()

                                  Strategy        Benchmark SPY
start                  1994-01-07 00:00:00  1994-01-07 00:00:00
end                    2021-12-31 00:00:00  2021-12-31 00:00:00
rf                                     0.0                  0.0
total_return                   1600.250781            15.889948
cagr                              0.301737             0.106303
max_drawdown                     -0.168972             -0.54613
calmar                            1.785728             0.194647
mtd                               0.023333             0.038474
three_month                       0.080918             0.073706
six_month                         0.127014             0.120826
ytd                               0.380719             0.304312
one_year                          0.380719             0.304312
three_year                        0.399576             0.256547
five_year                         0.361131             0.180282
ten_year                          0.3159

In [105]:
df = pd.read_csv('data.csv',parse_dates = True) #get_weekly('SPY').reset_index()

df['Date'] = pd.to_datetime(df['date'])
spy = get_weekly('SPY').reset_index()
benchmark = df.merge(spy,how = "left",on="Date")
to_store = pd.DataFrame()
spy_data = benchmark['Close'].pct_change()
cumulative_ret = (spy_data + 1).cumprod() -1
to_store['date'] = df['Date'].values[:]
to_store['equity_curve'] = df['nav'].values[:]
to_store['benchmark'] = benchmark['Close']
to_store['year'] = to_store['date'].dt.year
time_mask = (1994<=to_store['year']) & (to_store['year']<=2021)
to_store = to_store[time_mask]
to_store.to_csv('res1.csv')



In [330]:
monthly = pd.read_csv('monthly_table.csv')

In [331]:
df['year'] = pd.to_datetime(df['date']).dt.year

In [332]:
monthly['Benchmark SPY%'] =  [1.15,37.22,22.74,33.06,28.35,20.86,-9.15,-11.86,-22.12,28.39,10.75,4.79,15.69,5.39,-36.97,26.42,14.93,1.68,14.21,31.44,11.39,-0.73,11.80,19.42,-4.45,31.29,18.25,15.25]


In [333]:
cash = df.loc[(df['quantity']>0) | (df['buy']>0)].groupby('year')['quantity'].count()

In [334]:
c = 100 - cash*100/52
monthly['%Days in Cash'] =  list(c.round(2))[1:]

In [335]:
lev = df.loc[(df['leverage']>0) ].groupby('year')['leverage'].count()

In [336]:
l = (lev*100/52).round(2)

In [337]:
list(l.index
    ),l

([1993,
  1995,
  1996,
  1997,
  1998,
  1999,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2009,
  2010,
  2012,
  2013,
  2014,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021],
 year
 1993    17.31
 1995    21.15
 1996     7.69
 1997    13.46
 1998     3.85
 1999    21.15
 2001     1.92
 2002     1.92
 2003    23.08
 2004     3.85
 2005     3.85
 2006     7.69
 2009    30.77
 2010     9.62
 2012     7.69
 2013     7.69
 2014     3.85
 2016    11.54
 2017    17.31
 2018    13.46
 2019     5.77
 2020     7.69
 2021    13.46
 Name: leverage, dtype: float64)

In [338]:
monthly['%Days in leverage'] =[0,
 21.15,
 7.69,
 13.46,
 3.85,
 21.15,
 0,
 1.92,
 1.92,
 23.08,
 3.85,
 3.85,

 7.69,
 0,
 0,
 30.77,
 9.62,
 0,
 7.69,
 7.69,
 3.85,
 0,
 11.54,
 17.31,
 13.46,
 5.77,
 7.69,
 13.46]

In [339]:
monthly

,date,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Strategy YTD%,Benchmark SPY%,%Days in Cash,%Days in leverage
0,1994,0.00,-0.73,0.91,0.52,-1.28,0.00,3.01,-0.27,0.00,3.51,0.00,0.80,6.55,1.15,65.38,0.00
1,1995,0.00,0.00,0.00,0.00,8.94,2.30,0.00,0.00,17.41,0.00,1.84,0.00,33.26,37.22,75.00,21.15
2,1996,1.58,4.34,0.88,1.16,0.00,0.44,-5.05,3.92,4.70,2.41,16.01,5.50,40.68,22.74,53.85,7.69
3,1997,8.53,0.00,-3.00,-0.61,11.26,5.35,14.20,3.85,2.40,-0.30,2.75,3.46,57.89,33.06,55.77,13.46
4,1998,3.92,0.00,0.00,0.00,0.00,1.83,0.00,0.00,4.81,0.00,0.00,14.12,26.57,28.35,84.62,3.85
5,1999,0.00,-12.83,23.14,1.69,0.40,1.12,7.66,0.86,0.60,4.21,0.00,14.00,43.84,20.86,57.69,21.15
6,2000,0.00,0.00,5.09,0.00,0.00,7.12,0.00,2.61,0.00,-3.02,2.51,0.17,15.03,-9.15,75.00,0.00
7,2001,3.57,-2.97,-5.34,7.78,1.84,-0.20,-1.46,0.93,0.00,5.63,4.86,0.00,14.81,-11.86,38.46,1.92
8,2002,5.58,0.00,5.22,0.00,0.00,0.00,0.00,7.41,0.00,11.63,4.19,0.00,38.78,-22.12,73.08,1.92
9,2003,0.00,-0.33,2.54,0.51,0.00,0.00,10.53,4.48,3.58,2.52,0.00,1.59,27.97,28.39,55.77,23.08


In [340]:
monthly.to_csv('monthly_table.csv')

In [163]:
monthly.to_csv('monthly_table.csv')

In [31]:
generate_report()

                                  Strategy        Benchmark SPY
start                  1994-01-07 00:00:00  1994-01-07 00:00:00
end                    2021-12-31 00:00:00  2021-12-31 00:00:00
rf                                     0.0                  0.0
total_return                   1600.250781            15.889951
cagr                              0.301737             0.106303
max_drawdown                     -0.168972             -0.54613
calmar                            1.785728             0.194647
mtd                               0.023333             0.038474
three_month                       0.080918             0.073706
six_month                         0.127014             0.120826
ytd                               0.380719             0.304312
one_year                          0.380719             0.304312
three_year                        0.399576             0.256547
five_year                         0.361131             0.180282
ten_year                          0.3159

In [9]:
df

,Unnamed: 0,date,pid,nav,pnl,change,y_pred,y_pred2,y_pred3,y_pred4,...,200MA_prev,macdflattens_prev,dslow_flattens_prev,macdpositive_prev,macdlongpositive_prev,dstrend_prev,dslowtrend_prev,m_prev,d_prev,Date
0,0,1993-07-30,0,1.000000e+04,0.000000,0.279490,0,0,0.0,0,...,0,0,0,1.0,1.0,0,0,0.0,0.0,1993-07-30
1,0,1993-08-06,1,1.000000e+04,0.000000,0.278762,0,0,0.0,1,...,0,0,0,1.0,1.0,1,1,0.0,1.0,1993-08-06
2,0,1993-08-13,1,1.013866e+04,1.386620,0.347390,1,1,1.0,1,...,0,0,0,1.0,1.0,1,1,0.0,1.0,1993-08-13
3,0,1993-08-20,2,1.013866e+04,0.000000,1.523677,1,0,0.0,0,...,0,0,0,1.0,1.0,1,1,1.0,1.0,1993-08-20
4,0,1993-08-27,2,1.013866e+04,0.000000,0.954907,0,0,0.0,0,...,0,1,0,1.0,1.0,1,1,1.0,1.0,1993-08-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,0,2021-12-03,296,1.751893e+07,0.000000,-1.209231,0,0,1.0,0,...,1,1,1,1.0,1.0,0,0,0.0,0.0,2021-12-03
1480,0,2021-12-10,296,1.751893e+07,0.000000,3.819852,0,0,0.0,0,...,1,0,0,1.0,1.0,0,0,0.0,0.0,2021-12-10
1481,0,2021-12-17,297,1.751893e+07,0.000000,-1.965916,0,0,0.0,1,...,1,1,0,1.0,1.0,0,0,0.0,0.0,2021-12-17
1482,0,2021-12-24,297,1.792769e+07,2.333246,2.333271,1,0,1.0,0,...,1,1,0,1.0,1.0,0,0,0.0,0.0,2021-12-24


### Trade statistics

In [ ]:
len(a.loc[a['pnl']>0])
len(a.loc[a['pnl']<0])
len(a.loc[(a['quantity']>0) | (a['sell']>0) ])

In [682]:
# ok now what about some performance metrics?
# to_store = to_store.set_index('date')
prices = to_store[['equity_curve','benchmark']]
stats = prices.calc_stats().stats
# stats.display()

In [683]:
stats = pd.DataFrame(stats).round(2)

In [684]:
stats.to_csv('statistics.csv')

In [685]:
temp_stats = stats.to_dict()

In [686]:
stats = (stats[2:])
stats['equity_curve'] = ((stats['equity_curve']*100))#.round(2) #.astype('str') + '%'
stats['benchmark'] = ((stats['benchmark']*100))#.round(2) #.astype('str') + '%'

In [695]:
# stats = stats.reset_index()
kpi_table = stats[['equity_curve']]
# kpi_table = kpi_table.set_index('index')
kpi_table.index.name = 'KPI'
kpi_table.columns = ['Value']#,'Benchmark']
# kpi_table = kpi_table.set_index('KPI')
print(kpi_table)
kpi_table.loc['Period'] = 1 #'1994-2021'
kpi_table2 = kpi_table.loc[['Period','total_return','cagr',
                           'yearly_vol','max_drawdown','avg_drawdown']]#.astype(float)
kpi_table2 = kpi_table2.rename(index={'avg_drawdown':'Avg. drawdown','total_return':'Total return','max_drawdown':'Max drawdown','yearly_vol': "Yearly volatility", "avg_drawdown_days": "Avg drawdown duration in days"})


kpi_table2.index.name = 'KPI'

kpi_table2['Value'] = pd.Series(["{0:.2f}%".format(val) for val in kpi_table2['Value']], index = kpi_table2.index)

kpi_table2.loc['Period'] = '1994-2021'

kpi_table2.loc['Win/Loss ratio'] =  '93.28%'

# kpi_table2.loc['Total return'] = kpi_table2.loc['Total return']/100
# k2 = round(float(kpi_table.loc['calmar']/100),2)

# kpi_table2.loc['calmar'] = round(k2,2)

#add days in cash

                              Value
KPI                                
rf                              0.0
total_return           160025.07813
cagr                      30.173734
max_drawdown             -16.897163
calmar                   178.572782
mtd                        2.333256
three_month                8.091789
six_month                 12.701442
ytd                       38.071898
one_year                  38.071898
three_year                39.957582
five_year                 36.113133
ten_year                  31.591238
incep                     30.173734
daily_sharpe                    NaN
daily_sortino                   NaN
daily_mean                      NaN
daily_vol                       NaN
daily_skew                      NaN
daily_kurt                      NaN
best_day                        NaN
worst_day                       NaN
monthly_sharpe           181.709886
monthly_sortino          749.681635
monthly_mean              27.803724
monthly_vol               15

In [696]:
kpi_table2

,Value
KPI,
Period,1994-2021
Total return,160025.08%
cagr,30.17%
Yearly volatility,22.83%
Max drawdown,-16.90%
Avg. drawdown,-2.40%
Win/Loss ratio,93.28%


In [697]:
kpi_table2.to_csv('main_kpi.csv')

In [644]:
stats = stats.rename(index={'calmar':'Calmar','total_return':'Total return','cagr':'CAGR','max_drawdown':'Max drawdown','mtd': 'MTD','three_month':'Return last 3 months','six_month':'Return last 6 months','ytd':'YTD','one_year': "Return last 1 year",'three_year': "Return last 3 years", "five_year": "Returns last 5 years",'ten_year':'Returns last 10 years','incep':'Returns since inception'})
stats = stats.rename(index={'monthly_sharpe':'Monthly sharpe','monthly_sortino':'Monthly sortino','monthly_mean':'Monthly mean','monthly_vol':'Monthly volatility','monthly_skew':'Monthly skewness','monthly_kurt': 'Monthly kurtosis','best_month':'Best month','worst_month':'Worst month'})
stats = stats.rename(index={'yearly_sharpe':'Yearly sharpe','yearly_sortino':'Yearly sortino','yearly_mean':'Yearly mean','yearly_vol':'Yearly volatility','yearly_skew':'Yearly skew','yearly_kurt':'Yearly kurtosis'})
stats = stats.rename(index={'avg_drawdown':'Avg. drawdown','avg_drawdown_days':'Avg. drawdown duration in days','avg_up_month':'Avg. up month','avg_down_month':'Avg. down month','best_year':'Best year','worst_year':'Worst year','win_year_perc': '%Positive years','twelve_month_win_perc':'%Positive months last year'})

stats.columns = ['Strategy','Benchmark SPY']

kpi2 = stats.loc[['Total return', 'CAGR', 'Max drawdown', 'Calmar',
       'MTD', 'Return last 3 months', 'Return last 6 months', 'YTD',
       'Return last 1 year', 'Return last 3 years', 'Returns last 5 years',
       'Returns last 10 years', 'Returns since inception']].astype(float).round(2)
kpi3 = stats.loc[['Monthly sharpe', 'Monthly sortino',
       'Monthly mean', 'Monthly volatility', 'Monthly skewness',
       'Monthly kurtosis', 'Best month', 'Worst month']].astype(float).round(2)
kpi4 = stats.loc[['Yearly sharpe',
       'Yearly sortino', 'Yearly mean', 'Yearly volatility', 'Yearly skew',
       'Yearly kurtosis', 'Best year', 'Worst year']].astype(float).round(2)
kpi5 = stats.loc[['Avg. drawdown',
       'Avg. drawdown duration in days', 'Avg. up month', 'Avg. down month',
       '%Positive years', '%Positive months last year']].astype(float).round(2)

kpi2.loc['Calmar'] = (kpi2.loc['Calmar']/100).astype(float)
kpi3.loc['Monthly sortino'] = (kpi3.loc['Monthly sortino']/100).astype(float)
kpi3.loc['Monthly sharpe'] = (kpi3.loc['Monthly sharpe']/100).astype(float)
kpi3.loc['Monthly skewness'] = (kpi3.loc['Monthly skewness']/100).astype(float)
kpi3.loc['Monthly kurtosis'] = (kpi3.loc['Monthly kurtosis']/100).astype(float)


kpi4.loc['Yearly sortino'] = (kpi4.loc['Yearly sortino']/100).astype(float)
kpi4.loc['Yearly sharpe'] = (kpi4.loc['Yearly sharpe']/100).astype(float)
kpi4.loc['Yearly skew'] = (kpi4.loc['Yearly skew']/100).astype(float)
kpi4.loc['Yearly kurtosis'] = (kpi4.loc['Yearly kurtosis']/100).astype(float)


kpi5.loc['Avg. drawdown duration in days'] = (kpi5.loc['Avg. drawdown duration in days']/100).astype(float)

In [645]:
kpi4

,Strategy,Benchmark SPY
KPI,,
Yearly sharpe,1.4296,0.6847
Yearly sortino,inf,1.4423
Yearly mean,32.6300,12.8000
Yearly volatility,22.8300,18.6900
Yearly skew,2.9289,-1.0253
Yearly kurtosis,11.9895,1.0355
Best year,128.4600,38.0500
Worst year,5.6400,-39.4000


In [615]:
kpi2.to_csv('kpi2.csv')
kpi3.to_csv('kpi3.csv')
kpi4.to_csv('kpi4.csv')
kpi5.to_csv('kpi5.csv')

In [608]:
kpi4

,equity_curve,benchmark
KPI,,
Yearly sharpe,1.4296,0.6847
Yearly sortino,inf,1.4423
Yearly mean,32.6300,12.8000
Yearly volatility,22.8300,18.6900
Yearly skew,292.8900,-102.5300
Yearly kurtosis,1198.9500,103.5500
Best year,128.4600,38.0500
Worst year,5.6400,-39.4000


In [706]:
df

,Unnamed: 0,date,pid,nav,pnl,change,y_pred,y_pred2,y_pred3,y_pred4,...,macdflattens_prev,dslow_flattens_prev,macdpositive_prev,macdlongpositive_prev,dstrend_prev,dslowtrend_prev,m_prev,d_prev,Date,year
0,0,1993-07-30,1,1.000000e+04,0.000000,0.279620,0,0,0.0,1,...,0,0,1.0,1.0,0,0,0.0,0.0,1993-07-30,1993
1,0,1993-08-06,1,1.011123e+04,0.000000,0.278689,0,0,0.0,0,...,0,0,1.0,1.0,0,0,0.0,0.0,1993-08-06,1993
2,0,1993-08-13,1,1.025027e+04,0.000000,0.347419,0,1,0.0,1,...,0,0,1.0,1.0,0,0,0.0,0.0,1993-08-13,1993
3,0,1993-08-20,1,1.086214e+04,0.000000,1.523555,0,0,0.0,0,...,0,0,1.0,1.0,0,0,0.0,0.0,1993-08-20,1993
4,0,1993-08-27,1,1.125152e+04,0.000000,0.955027,0,0,0.0,0,...,0,0,1.0,1.0,0,0,0.0,0.0,1993-08-27,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,0,2021-12-03,296,1.968624e+07,0.000000,-1.209231,0,0,1.0,0,...,1,1,1.0,1.0,0,0,0.0,0.0,2021-12-03,2021
1480,0,2021-12-10,296,1.968624e+07,0.000000,3.819852,0,0,1.0,0,...,0,0,1.0,1.0,0,0,0.0,0.0,2021-12-10,2021
1481,0,2021-12-17,297,1.968624e+07,0.000000,-1.965916,0,0,0.0,1,...,1,0,1.0,1.0,0,0,0.0,0.0,2021-12-17,2021
1482,0,2021-12-24,297,2.014557e+07,2.333256,2.333271,1,0,1.0,0,...,1,0,1.0,1.0,0,0,0.0,0.0,2021-12-24,2021


In [713]:
prices = df[['date','nav']]
prices['date'] = pd.to_datetime(prices['date'])
prices = prices.set_index('date')
obj = prices #prices etc

yearly_prices = obj.resample('A').last()





nperiods = 1
yearly_returns = yearly_prices / yearly_prices.shift(1) - 1

returns = yearly_returns
print(returns+1)
er = np.power(1 + returns, 1. / nperiods) - 1.0 # returns #.to_excess_returns(rf, nperiods=nperiods)
#print(er)





res = er.mean() / er[er < 0].std()

res * np.sqrt(nperiods) #*100




                 nav
date                
1993-12-31       NaN
1994-12-31  1.065467
1995-12-31  1.332648
1996-12-31  1.406751
1997-12-31  1.578883
1998-12-31  1.265711
1999-12-31  1.438399
2000-12-31  1.150303
2001-12-31  1.148059
2002-12-31  1.387835
2003-12-31  1.279678
2004-12-31  1.075459
2005-12-31  1.177683
2006-12-31  1.208432
2007-12-31  1.204103
2008-12-31  1.255803
2009-12-31  2.284593
2010-12-31  1.373026
2011-12-31  1.056372
2012-12-31  1.319938
2013-12-31  1.252066
2014-12-31  1.208179
2015-12-31  1.147094
2016-12-31  1.366651
2017-12-31  1.418817
2018-12-31  1.254384
2019-12-31  1.418270
2020-12-31  1.421296
2021-12-31  1.380719


nav   NaN
dtype: float64

In [26]:
temp_stats

{'equity_curve': {'start': Timestamp('1994-01-07 00:00:00'),
  'end': Timestamp('2021-12-31 00:00:00'),
  'rf': 0.0,
  'total_return': 1600.0453056313274,
  'cagr': 0.3017313735822811,
  'max_drawdown': -0.16896938865488553,
  'calmar': 1.7857161938282062,
  'mtd': 0.023332459831210794,
  'three_month': 0.08091684551693645,
  'six_month': 0.12701295676437896,
  'ytd': 0.38071474877134803,
  'one_year': 0.38071474877134803,
  'three_year': 0.3995730518599858,
  'five_year': 0.3611289724596345,
  'ten_year': 0.3159094998398251,
  'incep': 0.3017313735822811,
  'daily_sharpe': nan,
  'daily_sortino': nan,
  'daily_mean': nan,
  'daily_vol': nan,
  'daily_skew': nan,
  'daily_kurt': nan,
  'best_day': nan,
  'worst_day': nan,
  'monthly_sharpe': 1.8172893358007165,
  'monthly_sortino': 7.500286161275699,
  'monthly_mean': 0.27802974989187784,
  'monthly_vol': 0.1529914606412275,
  'monthly_skew': 2.154657917790354,
  'monthly_kurt': 7.832333308972752,
  'best_month': 0.2708732804038134,
  

In [84]:
returns = to_store[['equity_curve','benchmark']].pct_change()

ffn.core.calc_sortino_ratio(returns, rf=0, nperiods=None, annualize=True)*100

equity_curve    65.521155
benchmark       14.768506
dtype: float64

In [92]:
ffn.core.calc_sharpe(returns, rf=0, nperiods=None, annualize=True)

equity_curve    0.231138
benchmark       0.092823
dtype: float64

In [ ]:
ffn.core.to_excess_returns(returns, rf, nperiods=None)

HI Nicole and Yannick, hope you had a refreshing break in Paris. During the past week a couple of people have requested more information about system characteristics: a) number of trades in a year; b) number of winning and losing trades c) number of months that the system sustained monthly losses  more than 5%                                                                               I also would be interested  in looking any results or conclusions on a day trading system                                                                       Finally, does a day trading approach work on vix or uvxy?                        I know it's a lot of different questions/ issues but to there need to be answers even for ourselves.                                                              see you both on Monday at lunch

In [646]:
trades = pd.DataFrame()

In [647]:
number = len(df.loc[df['sell']>0])

In [648]:
number

149

In [692]:
profitable = len(df.loc[df['pnl']>0])

In [650]:
profitable

139

In [651]:
loss =  len(df.loc[df['pnl']<0])

In [693]:
loss

10

In [653]:
per_positive_trades = profitable*100/number

In [654]:
per_negative_trades = loss*100/number

In [655]:
avg_trades_year = round(number/(2021-1994))

In [671]:
trades = pd.DataFrame(df.loc[df['pnl']!=0]['pnl'].astype(float).describe().round(3))

In [672]:
trades.loc['%Positive trades'] = round(per_positive_trades,3)

In [673]:
trades.loc['%Negative trades'] = round(per_negative_trades,3)
trades.loc['Avg. trades per year'] = avg_trades_year
trades.loc['% of trades leveraged'] =   round((df.loc[(df['quantity']>0) & (df['buy']>0) & (df['leverage']>0)]['date'].count()/ number)*100,2)

In [694]:
profitable*100/number

93.28859060402685

In [674]:
trades = trades.rename(index={'count':'Total trades','std': 'Standard deviation','mean':'Avg. profit per trade','min':'Max loss','max':'Max profit','25%': "Profits 25% quartile",'75%': "Profits 75% quartile", "50%": "Profits 50% quartile"})
tr = trades.round(2)

In [675]:
tr

,pnl
Total trades,149.00
Avg. profit per trade,5.36
Standard deviation,5.16
Max loss,-0.94
Profits 25% quartile,1.83
Profits 50% quartile,3.74
Profits 75% quartile,7.47
Max profit,27.09
%Positive trades,93.29
%Negative trades,6.71


In [676]:
t1 = trades.loc[['Avg. profit per trade', 'Max loss','Max profit','Profits 25% quartile','Profits 50% quartile','Profits 75% quartile','%Positive trades','%Negative trades','% of trades leveraged']].astype('str')  + '%'

In [678]:
t2 = trades.loc[['Total trades','Standard deviation','Avg. trades per year']]

In [679]:
trades_final = t2.append(t1)
trades.columns = ['Value']
trades_final.to_csv('trades.csv')

In [320]:
returns['date'] = pd.to_datetime(to_store['date'])
returns = returns.set_index('date')
return_series = (returns/ float(1))
returns_df_m = pd.DataFrame((return_series + 1).resample('M').prod() - 1)

AttributeError: 'NaTType' object has no attribute 'normalize'

In [287]:
mret = returns_df_m['equity_curve']*100

In [288]:
mret

date
1994-01-31    0.000000
1994-02-28   -0.728615
1994-03-31    0.912156
1994-04-30    0.522309
1994-05-31   -1.281515
                ...   
2021-08-31    1.554034
2021-09-30    0.568643
2021-10-31    3.455630
2021-11-30    2.099070
2021-12-31    2.333256
Freq: M, Name: equity_curve, Length: 336, dtype: float64